<a href="https://colab.research.google.com/github/shailendrayadav05/IITKshailendra/blob/main/Taskmanager_userauth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import json
import os

# File names
USERS_FILE = 'users.json'
TASKS_DIR = 'tasks'

# Ensure tasks directory exists
if not os.path.exists(TASKS_DIR):
    os.makedirs(TASKS_DIR)

def hash_password(password):
    """Hash a password using SHA-256."""
    return hashlib.sha256(password.encode()).hexdigest()

def load_users():
    """Load users from file."""
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_users(users):
    """Save users to file."""
    with open(USERS_FILE, 'w') as f:
        json.dump(users, f)

def register():
    """Register a new user."""
    users = load_users()
    username = input("Enter username: ").strip()
    if username in users:
        print("Username already exists.")
        return False
    password = input("Enter password: ").strip()
    users[username] = hash_password(password)
    save_users(users)
    print("Registration successful.")
    return True

def login():
    """Login user."""
    users = load_users()
    username = input("Enter username: ").strip()
    password = input("Enter password: ").strip()
    if username in users and users[username] == hash_password(password):
        print("Login successful.")
        return username
    print("Invalid credentials.")
    return None

def load_tasks(username):
    """Load tasks for a user."""
    task_file = os.path.join(TASKS_DIR, f"{username}_tasks.json")
    if os.path.exists(task_file):
        with open(task_file, 'r') as f:
            return json.load(f)
    return []

def save_tasks(username, tasks):
    """Save tasks for a user."""
    task_file = os.path.join(TASKS_DIR, f"{username}_tasks.json")
    with open(task_file, 'w') as f:
        json.dump(tasks, f)

def get_next_id(tasks):
    """Get next task ID."""
    if not tasks:
        return 1
    return max(task['id'] for task in tasks) + 1

def add_task(username):
    """Add a new task."""
    tasks = load_tasks(username)
    description = input("Enter task description: ").strip()
    task_id = get_next_id(tasks)
    tasks.append({'id': task_id, 'description': description, 'status': 'Pending'})
    save_tasks(username, tasks)
    print("Task added successfully.")

def view_tasks(username):
    """View all tasks."""
    tasks = load_tasks(username)
    if not tasks:
        print("No tasks found.")
        return
    for task in tasks:
        print(f"ID: {task['id']}, Description: {task['description']}, Status: {task['status']}")

def mark_completed(username):
    """Mark a task as completed."""
    tasks = load_tasks(username)
    try:
        task_id = int(input("Enter task ID to mark as completed: ").strip())
    except ValueError:
        print("Invalid ID.")
        return
    for task in tasks:
        if task['id'] == task_id:
            task['status'] = 'Completed'
            save_tasks(username, tasks)
            print("Task marked as completed.")
            return
    print("Task not found.")

def delete_task(username):
    """Delete a task."""
    tasks = load_tasks(username)
    try:
        task_id = int(input("Enter task ID to delete: ").strip())
    except ValueError:
        print("Invalid ID.")
        return
    tasks = [task for task in tasks if task['id'] != task_id]
    save_tasks(username, tasks)
    print("Task deleted successfully.")

def display_menu():
    """Display task manager menu."""
    print("\nTask Manager Menu:")
    print("1. Add a Task")
    print("2. View Tasks")
    print("3. Mark a Task as Completed")
    print("4. Delete a Task")
    print("5. Logout")

def task_manager(username):
    """Main task manager loop."""
    while True:
        display_menu()
        choice = input("Enter your choice (1-5): ").strip()
        if choice == '1':
            add_task(username)
        elif choice == '2':
            view_tasks(username)
        elif choice == '3':
            mark_completed(username)
        elif choice == '4':
            delete_task(username)
        elif choice == '5':
            print("Logging out.")
            break
        else:
            print("Invalid choice.")

def main():
    """Main program entry."""
    while True:
        print("\n1. Register")
        print("2. Login")
        print("3. Exit")
        choice = input("Enter choice (1-3): ").strip()
        if choice == '1':
            register()
        elif choice == '2':
            username = login()
            if username:
                task_manager(username)
        elif choice == '3':
            print("Exiting program.")
            break
        else:
            print("Invalid choice.")

if __name__ == "__main__":
    main()